## Сравнение POS-теггеров

In [1]:
import nltk
from pymystem3 import Mystem
m = Mystem()
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize

In [2]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

Текст пришлось написать самостоятельно. Я постаралась добавить в него некоторое количество омоформ, на которых (в силу одинакового написания разных частей речи) должна ломаться разметка.

In [3]:
text = """Сегодня мать и дочь собрались печь пироги. Для первого пирога, яблочного, начинка уже была готова. Поэтому мама начала заниматься тестом, а дочери сказала: "Мой продукты, только в этот раз не отжимай зелень, очень тебя прошу, просто оставь в сите, чтобы вода стекла." Дочь недовольно фыркнула, но сделала как сказали. Когда капуста и зелень были вымыты, а после и нарезаны, дочь спросила у матери: "В какую миску это ссыпать?" И получила ответ "в ту, большую, из стекла". Пока девушка продолжила заниматься начинкой для капустного пирога, мама уже поставила первый в печь. Теперь она ходила по комнате и что-то искала.
- Милая, ты не видела мой телефон? Я хочу засечь время.
- Нет, мам. Давай сейчас я поставлю таймер, а телефон потом поищем.
Выставив время, они вернулись ко второму пирогу. В этот момент пришёл домой отец.
- Как у вас тут вкусно пахнет. Хорошо, что сегодня мне дали уйти пораньше!
Тем временем задребезжал таймер.
- Доча, посмотри, если края стали золотистыми, то можно вынимать.
- Пусть ещё постоит. Мы почти закончили капустник, будем его ставить - вынем яблочный.
- Хорошо, значит простой нагретой духовки не придётся устраивать.
Через некоторое время с готовкой было покончено. Семья собралась за столом и делились впечатлениями минувшей рабочей недели. Простой пятничный вечер казался почти праздником, настолько вкусные пироги получились."""
doc = Doc(text)
pm_text = simple_word_tokenize(text)

Сначала размечаем каждым из разметчиков (избавляемся от пунктуации, где это необходимо) и проверяем, что количество слов получилось одинаковое
(для пайморфи сохранён как вариант с первым разбором, так и с неснятой омонимией)

In [4]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
natasha_pos = []
for w in doc.tokens:
    natasha_pos += [[w.text, w.pos]]
print(natasha_pos[:20])

[['Сегодня', 'ADV'], ['мать', 'NOUN'], ['и', 'CCONJ'], ['дочь', 'NOUN'], ['собрались', 'VERB'], ['печь', 'VERB'], ['пироги', 'NOUN'], ['.', 'PUNCT'], ['Для', 'ADP'], ['первого', 'ADJ'], ['пирога', 'NOUN'], [',', 'PUNCT'], ['яблочного', 'ADJ'], [',', 'PUNCT'], ['начинка', 'NOUN'], ['уже', 'ADV'], ['была', 'AUX'], ['готова', 'ADJ'], ['.', 'PUNCT'], ['Поэтому', 'ADV']]


In [5]:
for i in range(2):
    for w in natasha_pos:
        if w[1] == 'PUNCT':
            natasha_pos.remove(w)
print(len(natasha_pos), natasha_pos[:20])

205 [['Сегодня', 'ADV'], ['мать', 'NOUN'], ['и', 'CCONJ'], ['дочь', 'NOUN'], ['собрались', 'VERB'], ['печь', 'VERB'], ['пироги', 'NOUN'], ['Для', 'ADP'], ['первого', 'ADJ'], ['пирога', 'NOUN'], ['яблочного', 'ADJ'], ['начинка', 'NOUN'], ['уже', 'ADV'], ['была', 'AUX'], ['готова', 'ADJ'], ['Поэтому', 'ADV'], ['мама', 'NOUN'], ['начала', 'VERB'], ['заниматься', 'VERB'], ['тестом', 'NOUN']]


In [6]:
pm_pos = []
pm_pos_o = []
for w in pm_text:
    ana = morph.parse(w)
    pm_pos += [[ana[0].word, ana[0].tag.POS]]
    pm_pos_part = []
    for v in ana:
        if v.tag.POS not in pm_pos_part:
            pm_pos_part += [v.tag.POS]
    pm_pos_o += [[ana[0].word, pm_pos_part]]
print(len(pm_pos))


268


In [7]:
for i in range(2):
    for w in pm_pos:
        if w[1] == None:
            pm_pos.remove(w)

    for w in pm_pos_o:
        if w[1] == [None]:
            pm_pos_o.remove(w)
print(len(pm_pos), pm_pos[:20])
print(len(pm_pos_o), pm_pos_o[:20])

205 [['сегодня', 'ADVB'], ['мать', 'NOUN'], ['и', 'CONJ'], ['дочь', 'NOUN'], ['собрались', 'VERB'], ['печь', 'INFN'], ['пироги', 'NOUN'], ['для', 'PREP'], ['первого', 'NOUN'], ['пирога', 'NOUN'], ['яблочного', 'ADJF'], ['начинка', 'NOUN'], ['уже', 'ADVB'], ['была', 'VERB'], ['готова', 'ADJS'], ['поэтому', 'ADVB'], ['мама', 'NOUN'], ['начала', 'NOUN'], ['заниматься', 'INFN'], ['тестом', 'NOUN']]
205 [['сегодня', ['ADVB']], ['мать', ['NOUN']], ['и', ['CONJ', 'INTJ', 'PRCL', 'NOUN']], ['дочь', ['NOUN']], ['собрались', ['VERB']], ['печь', ['INFN', 'NOUN']], ['пироги', ['NOUN']], ['для', ['PREP', 'GRND']], ['первого', ['NOUN', 'ADJF']], ['пирога', ['NOUN']], ['яблочного', ['ADJF']], ['начинка', ['NOUN']], ['уже', ['ADVB', 'PRCL', 'NOUN', 'COMP']], ['была', ['VERB']], ['готова', ['ADJS']], ['поэтому', ['ADVB']], ['мама', ['NOUN']], ['начала', ['NOUN', 'VERB']], ['заниматься', ['INFN']], ['тестом', ['NOUN']]]


In [8]:
analyz = m.analyze(text)
mystem_pos = []
for w in analyz:
    if 'analysis' in w:
        pos = w['analysis'][0]['gr']
        pos = pos.split('=')[0].split(',')[0]
        mystem_pos += [[w['text'], pos]]
print(len(mystem_pos))

205


In [9]:
print(len(mystem_pos), mystem_pos[:20])

205 [['Сегодня', 'ADV'], ['мать', 'S'], ['и', 'CONJ'], ['дочь', 'S'], ['собрались', 'V'], ['печь', 'S'], ['пироги', 'S'], ['Для', 'PR'], ['первого', 'ANUM'], ['пирога', 'S'], ['яблочного', 'A'], ['начинка', 'S'], ['уже', 'ADV'], ['была', 'V'], ['готова', 'A'], ['Поэтому', 'ADVPRO'], ['мама', 'S'], ['начала', 'V'], ['заниматься', 'V'], ['тестом', 'S']]


Приводим теги к общему набору, который использовался в разметке стандарта

In [10]:
for w in natasha_pos:
    if w[1] == 'AUX':
        w[1] = 'VERB'
    elif w[1] in ['CCONJ','SCONJ']:
        w[1] = 'CONJ'
    elif w[1] == 'DET':
        w[1] = 'APRO'

In [11]:
print(natasha_pos[:20])

[['Сегодня', 'ADV'], ['мать', 'NOUN'], ['и', 'CONJ'], ['дочь', 'NOUN'], ['собрались', 'VERB'], ['печь', 'VERB'], ['пироги', 'NOUN'], ['Для', 'ADP'], ['первого', 'ADJ'], ['пирога', 'NOUN'], ['яблочного', 'ADJ'], ['начинка', 'NOUN'], ['уже', 'ADV'], ['была', 'VERB'], ['готова', 'ADJ'], ['Поэтому', 'ADV'], ['мама', 'NOUN'], ['начала', 'VERB'], ['заниматься', 'VERB'], ['тестом', 'NOUN']]


In [12]:
for w in pm_pos:
    if w[1] == 'ADVB':
        w[1] = 'ADV'
    elif w[1] in ['ADJF','ADJS']:
        w[1] = 'ADJ'
    elif w[1] in ['INFN','GRND','PRTF','PRTS']:
        w[1] = 'VERB'
    elif w[1] == 'NPRO':
        w[1] = 'PRON'
    elif w[1] == 'PREP':
        w[1] = 'ADP'
    elif w[1] == 'PRCL':
        w[1] = 'PART'

for w in pm_pos_o:
    for t in range(len(w[1])):
        if w[1][t] == 'ADVB':
            w[1][t] = 'ADV'
        elif w[1][t] in ['ADJF','ADJS']:
            w[1][t] = 'ADJ'
        elif w[1][t] in ['INFN','GRND','PRTF','PRTS']:
            w[1][t] = 'VERB'
        elif w[1][t] == 'NPRO':
            w[1][t] = 'PRON'
        elif w[1][t] == 'PREP':
            w[1][t] = 'ADP'
        elif w[1][t] == 'PRCL':
            w[1][t] = 'PART'

In [13]:
print(pm_pos[:20])
print(pm_pos_o[:20])

[['сегодня', 'ADV'], ['мать', 'NOUN'], ['и', 'CONJ'], ['дочь', 'NOUN'], ['собрались', 'VERB'], ['печь', 'VERB'], ['пироги', 'NOUN'], ['для', 'ADP'], ['первого', 'NOUN'], ['пирога', 'NOUN'], ['яблочного', 'ADJ'], ['начинка', 'NOUN'], ['уже', 'ADV'], ['была', 'VERB'], ['готова', 'ADJ'], ['поэтому', 'ADV'], ['мама', 'NOUN'], ['начала', 'NOUN'], ['заниматься', 'VERB'], ['тестом', 'NOUN']]
[['сегодня', ['ADV']], ['мать', ['NOUN']], ['и', ['CONJ', 'INTJ', 'PART', 'NOUN']], ['дочь', ['NOUN']], ['собрались', ['VERB']], ['печь', ['VERB', 'NOUN']], ['пироги', ['NOUN']], ['для', ['ADP', 'VERB']], ['первого', ['NOUN', 'ADJ']], ['пирога', ['NOUN']], ['яблочного', ['ADJ']], ['начинка', ['NOUN']], ['уже', ['ADV', 'PART', 'NOUN', 'COMP']], ['была', ['VERB']], ['готова', ['ADJ']], ['поэтому', ['ADV']], ['мама', ['NOUN']], ['начала', ['NOUN', 'VERB']], ['заниматься', ['VERB']], ['тестом', ['NOUN']]]


In [20]:
for w in mystem_pos:
    if w[1] in ['A', 'ANUM']:
        w[1] = 'ADJ'
    elif w[1] == 'S':
        w[1] = 'NOUN'
    elif w[1] == 'PR':
        w[1] = 'ADP'
    elif w[1] == 'SPRO':
        w[1] = 'PRON'
    elif w[1] == 'V':
        w[1] = 'VERB' 
    elif w[1] == 'ADVPRO':
        w[1] = 'ADV'

In [21]:
print(mystem_pos[:20])

[['Сегодня', 'ADV'], ['мать', 'NOUN'], ['и', 'CONJ'], ['дочь', 'NOUN'], ['собрались', 'VERB'], ['печь', 'NOUN'], ['пироги', 'NOUN'], ['Для', 'ADP'], ['первого', 'ADJ'], ['пирога', 'NOUN'], ['яблочного', 'ADJ'], ['начинка', 'NOUN'], ['уже', 'ADV'], ['была', 'VERB'], ['готова', 'ADJ'], ['Поэтому', 'ADV'], ['мама', 'NOUN'], ['начала', 'VERB'], ['заниматься', 'VERB'], ['тестом', 'NOUN']]


Импортируем таблицу со стандартом и смотрим, есть ли вхождение верных тегов по словам

In [16]:
import csv

In [17]:
with open('pos_text_hw2.csv', newline='', encoding='UTF-8') as f:
    standart = list(csv.reader(f))

for s in standart:
    s[1] = s[1].split(',')
print(standart[:20], len(standart))

[['Сегодня', ['ADV']], ['мать', ['NOUN']], ['и', ['CONJ']], ['дочь', ['NOUN']], ['собрались', ['VERB']], ['печь', ['VERB']], ['пироги', ['NOUN']], ['Для', ['ADP']], ['первого', ['ADJ']], ['пирога', ['NOUN']], ['яблочного', ['ADJ']], ['начинка', ['NOUN']], ['уже', ['ADV']], ['была', ['VERB']], ['готова', ['ADJ']], ['Поэтому', ['ADV']], ['мама', ['NOUN']], ['начала', ['VERB']], ['заниматься', ['VERB']], ['тестом', ['NOUN']]] 205


In [22]:
natasha_score = 0
pm_score = 0
pm_o_score = 0
mystem_score = 0
for s in range(len(standart)):
    if set(standart[s][1])&set([natasha_pos[s][1]]):
        natasha_score += 1
    else:
        print(standart[s], natasha_pos[s][1], 'natasha')
    if set(standart[s][1])&set([pm_pos[s][1]]):
        pm_score += 1
    else:
        print(standart[s], pm_pos[s][1], 'pymorphy')
    if set(standart[s][1])&set(pm_pos_o[s][1]):
        pm_o_score += 1
    else:
        print(standart[s], pm_pos_o[s][1], 'pymorphy with o')
    if set(standart[s][1])&set([mystem_pos[s][1]]):
        mystem_score += 1
    else:
        print(standart[s], mystem_pos[s][1], 'mystem')

['печь', ['VERB']] NOUN mystem
['первого', ['ADJ']] NOUN pymorphy
['начала', ['VERB']] NOUN pymorphy
['Мой', ['VERB']] APRO natasha
['Мой', ['VERB']] ADJ pymorphy
['Мой', ['VERB']] APRO mystem
['только', ['ADV']] PART natasha
['только', ['ADV']] PART pymorphy
['только', ['ADV']] ['PART', 'CONJ'] pymorphy with o
['только', ['ADV']] PART mystem
['этот', ['APRO']] ADJ pymorphy
['этот', ['APRO']] ['ADJ'] pymorphy with o
['просто', ['ADV']] PART natasha
['просто', ['ADV']] PART pymorphy
['просто', ['ADV']] PART mystem
['стекла', ['VERB']] NOUN natasha
['стекла', ['VERB']] NOUN pymorphy
['стекла', ['VERB']] NOUN mystem
['недовольно', ['ADV']] ADJ natasha
['недовольно', ['ADV']] ADJ pymorphy
['фыркнула', ['VERB']] NOUN natasha
['после', ['ADP']] ADV natasha
['и', ['PART']] CONJ natasha
['и', ['PART']] CONJ pymorphy
['и', ['PART']] CONJ mystem
['нарезаны', ['VERB']] NOUN natasha
['это', ['PRON']] PART pymorphy
['это', ['PRON']] PART mystem
['ссыпать', ['VERB']] NOUN natasha
['ту', ['APRO']] AD

Считаем точность разметки

In [23]:
acc_natasha = natasha_score/len(standart)
acc_pm = pm_score/len(standart)
acc_pm_o = pm_o_score/len(standart)
acc_mystem = mystem_score/len(standart)

print(acc_natasha, acc_pm, acc_pm_o, acc_mystem)

0.9073170731707317 0.8682926829268293 0.9512195121951219 0.9463414634146341


Pymorphy с неснятой омонимией, хоть и доказал, что чаще всего имеет достаточно типов разбора, чтобы один из них оказывался верным, тем не менее невозможен для дальнейшей работы, так как контекст не учитывает и, соответственно, предлагает невозможные комбинации рядом стоящих тегов

Мы же возьмём снимающий омонимию и находящийся на следующем месте по количеству верных ответов вариант - mystem
(этот итог был достаточно предсказуем, так как его способность снимать частеречную омонимию известна, пусть он и не со всеми случаями справился в этот раз)

Используя разметку mystem, которая показала себя с лучшей стороны, ищем би- и три-граммы

In [42]:
def chunker_3(tw_1, tw_2, tw_3, text_pos):
    chunk_list = []
    for w in range(len(text_pos)-2):
        if (text_pos[w][1] in tw_1 or text_pos[w][0] in tw_1) and (text_pos[w+1][1] in tw_2 or text_pos[w+1][0] in tw_2) and (text_pos[w+2][1] in tw_3 or text_pos[w+2][0] in tw_3):
            chunk = [text_pos[w][0], text_pos[w+1][0], text_pos[w+2][0]]
            chunk_list += [chunk]
    return chunk_list            

In [43]:
noun_conj_s = chunker_3('NOUN','и','NOUN', mystem_pos)

In [44]:
noun_conj_s

[['мать', 'и', 'дочь'], ['капуста', 'и', 'зелень']]

In [45]:
v_v_s = chunker_3('VERB', 'VERB', ['NOUN', 'ADJ', 'ADV'], mystem_pos)

In [46]:
v_v_s

[['начала', 'заниматься', 'тестом'],
 ['продолжила', 'заниматься', 'начинкой'],
 ['хочу', 'засечь', 'время'],
 ['поищем', 'Выставив', 'время'],
 ['дали', 'уйти', 'пораньше'],
 ['ставить', 'вынем', 'яблочный'],
 ['было', 'покончено', 'Семья']]

In [49]:
def chunker_2(tw_1, tw_2, text_pos):
    chunk_list = []
    for w in range(len(text_pos)-1):
        if (text_pos[w][1] in tw_1 or text_pos[w][0] in tw_1) and (text_pos[w+1][1] in tw_2 or text_pos[w+1][0] in tw_2):
            chunk = [text_pos[w][0], text_pos[w+1][0]]
            chunk_list += [chunk]
    return chunk_list
    

In [50]:
adj_noun = chunker_2('ADJ', 'NOUN', mystem_pos)

In [51]:
adj_noun

[['первого', 'пирога'],
 ['яблочного', 'начинка'],
 ['капустного', 'пирога'],
 ['второму', 'пирогу'],
 ['рабочей', 'недели'],
 ['пятничный', 'вечер'],
 ['вкусные', 'пироги']]